In [0]:
# Mount ADLS Gen2
# Required each time the cluster is restarted which should be only on the first notebook as they run in order

tiers = ["bronze", "silver", "gold"]
adls_paths = {tier: f"abfss://{tier}@stockagedataengineering.dfs.core.windows.net/" for tier in tiers}

# Accessing paths
bronze_adls = adls_paths["bronze"]
silver_adls = adls_paths["silver"]
gold_adls = adls_paths["gold"] 

dbutils.fs.ls(bronze_adls)
dbutils.fs.ls(silver_adls)
dbutils.fs.ls(gold_adls)

[FileInfo(path='abfss://gold@stockagedataengineering.dfs.core.windows.net/Consultation.csv', name='Consultation.csv', size=14466626, modificationTime=1746460728000),
 FileInfo(path='abfss://gold@stockagedataengineering.dfs.core.windows.net/Date.csv', name='Date.csv', size=1908623, modificationTime=1746460724000)]

In [0]:
from pyspark.sql.functions import col, count, when
from pyspark.sql import functions as F
import os

# Dossiers Bronze et Silver
bronze_path = bronze_adls
silver_path = silver_adls

# Dictionnaire des noms traduits
translations = {
    
    "Allergy": "Allergie",
    "Date": "Date",
    "Disease": "Maladie",
    "Doctor": "Medecin",
    "Patient": "Patient",
    "Room": "Chambre",
    "Treatment": "Traitement",
    "TreatmentCost": "CoutTraitement",
    "Cost": "Cout",
    "Encounter": "Consultation",
    "Vitals": "SignesVitaux",
    "Patient_Allergy": "Patient_Allergie",
    "BridgeEncounterDoctor": "Docteur_consultation"
    # Ajoute d'autres si besoin
}

# Liste tous les fichiers CSV du dossier Bronze
csv_files = [f.path for f in dbutils.fs.ls(bronze_path) if f.path.endswith(".csv")]

for file_path in csv_files:
    # Lire le fichier
    df = spark.read.option("header", "true").csv(file_path)
    df = df.dropDuplicates()

    # Remplir les valeurs manquantes
    for column, dtype in df.dtypes:
        if dtype in ["double", "int", "float", "bigint"]:
            median = df.approxQuantile(column, [0.5], 0.001)[0]
            df = df.withColumn(column, when(col(column).isNull(), median).otherwise(col(column)))
        else:
            mode_df = df.groupBy(column).count().orderBy(F.desc("count")).first()
            if mode_df:
                mode_value = mode_df[0]
                df = df.withColumn(column, when(col(column).isNull(), mode_value).otherwise(col(column)))

    # Nettoyer le nom du fichier
    filename = os.path.basename(file_path)
    base_name = filename.replace(".csv", "")
    
    # Supprimer les préfixes "Dim_", "Fact_", etc.
    for prefix in ["Dim_", "Fact_", "Dim", "Fact"]:
        if base_name.startswith(prefix):
            base_name = base_name[len(prefix):]

    # Récupérer le nom traduit, sinon garder le nom sans préfixe
    translated_name = translations.get(base_name, base_name)
    final_filename = translated_name + ".csv"

    # Écriture dans un dossier temporaire
    tmp_output_path = silver_path + "_tmp_" + translated_name + "/"
    (df.coalesce(1)
       .write
       .mode("overwrite")
       .option("header", "true")
       .csv(tmp_output_path))

    # Trouver le fichier part-xxxxx.csv
    tmp_files = dbutils.fs.ls(tmp_output_path)
    part_file = [f.path for f in tmp_files if f.name.startswith("part-")][0]

    # Définir le chemin final dans Silver
    final_output_path = silver_path + final_filename

    # Déplacer et renommer
    dbutils.fs.mv(part_file, final_output_path)

    # Supprimer les fichiers temporaires
    dbutils.fs.rm(tmp_output_path, recurse=True)

print(" Tous les fichiers transformés et enregistrés avec noms traduits en français.")


✅ Tous les fichiers transformés et enregistrés avec noms traduits en français.


In [0]:
# from pyspark.sql.functions import col, count, when
# from pyspark.sql import functions as F
# import os

# # Dossiers Bronze et Silver
# bronze_path = bronze_adls
# silver_path = silver_adls

# # Liste de tous les fichiers CSV du dossier Bronze
# csv_files = [f.path for f in dbutils.fs.ls(bronze_path) if f.path.endswith('.csv')]

# for file_path in csv_files:
#     # Lire chaque fichier CSV
#     df = spark.read.option("header", "true").csv(file_path)

#     # Supprimer les doublons
#     df = df.dropDuplicates()

#     # Remplir les valeurs manquantes
#     for column, dtype in df.dtypes:
#         if dtype in ['double', 'int', 'float', 'bigint']:
#             median = df.approxQuantile(column, [0.5], 0.001)[0]
#             df = df.withColumn(column, when(col(column).isNull(), median).otherwise(col(column)))
#         else:
#             mode_df = df.groupBy(column).count().orderBy(F.desc("count")).first()
#             if mode_df:
#                 mode_value = mode_df[0]
#                 df = df.withColumn(column, when(col(column).isNull(), mode_value).otherwise(col(column)))

#     # Obtenir le nom de fichier original
#     filename = os.path.basename(file_path)

#     # Définir un chemin temporaire pour écrire
#     tmp_output_path = silver_path + "_tmp_" + filename.replace(".csv", "/")

#     # Ecrire le DataFrame en CSV (un seul fichier) dans un dossier temporaire
#     (df.coalesce(1)
#        .write
#        .mode("overwrite")
#        .option("header", "true")
#        .csv(tmp_output_path))

#     # Trouver le fichier part-*.csv dans le dossier temporaire
#     tmp_files = dbutils.fs.ls(tmp_output_path)
#     part_file = [f.path for f in tmp_files if f.name.startswith("part-")][0]

#     # Chemin final : fichier sous Silver avec le vrai nom
#     final_output_path = silver_path + filename

#     # Déplacer et renommer le fichier part-xxxxx.csv vers le chemin final
#     dbutils.fs.mv(part_file, final_output_path)

#     # Supprimer le dossier temporaire
#     dbutils.fs.rm(tmp_output_path, recurse=True)

# print("✅ Tous les fichiers transformés et enregistrés proprement sans fichiers parasites.")
